# Benchmarking the Mixed Meal Model in Julia

In [ ]:
# Base functionality
using Plots, Measures, Statistics

# ODE Solver
using DifferentialEquations

# Benchmarking
using BenchmarkTools

include("../MixedMealModel.jl")

## Simulation Benchmark
Below, we generate one test subject for simulation

In [ ]:
sample_person = (
    fasting_glucose = 5.,
    fasting_insulin = 18.,
    fasting_triglyceride = 1.3,
    fasting_NEFA = 0.33,
    body_weight = 84.2,
    meal_glucose = 75000.,
    meal_triglyceride = 60000.
)

parameters = InitialParameters(sample_person, k1 = 0.0164, k5=0.0564, 
k6 = 2.7341, k11 = 0.00035, k12 = 0.0822, tau_LPL = 187.88, 
k14 = 0.0392, k16 = 0.0135)

constants = Constants(parameters, sample_person);

In [ ]:
tspan::Tuple{Float64, Float64} = (0., 720.)
u0 = InitialValues(sample_person)
MealModel! = MixedMealModel(constants, sample_person)
System = ODEProblem(MealModel!, u0, tspan, parameters)

In [ ]:
Solution = solve(System);

BenchmarkResult = @benchmark solve(System) samples=10_000 seconds=30

In [ ]:
time = tspan[1]:0.01:tspan[2]
outputs =  ModelOutput(Solution, parameters, constants, sample_person; time=time);

#p_G_gut = plot(time, outputs.gut_glucose[:], title = "Gut Glucose", xlabel="time (min)", ylabel="Glucose (mmol/l)")
#p_G_liv = plot(time, outputs.hepatic_glucose_flux[:], title="Net Hepatic Glucose Flux", xlabel="time (min)", ylabel="Glucose (mmol/l)")
#p_G_tissue = plot(time, outputs.glucose_tissue_uptake[:], title = "Glucose Uptake into Tissue", xlabel="time (min)", ylabel="Glucose (mmol/l)")
p_G_pl = plot(time, outputs.plasma_glucose[:], title = "Plasma Glucose", xlabel="time (min)", ylabel="Glucose (mmol/l)")
p_I_pl = plot(time, outputs.plasma_insulin[:], title = "Plasma Insulin", xlabel="time (min)", ylabel="Insulin (uIU/ml)")
#p_TG_gut = plot(time, outputs.gut_TG[:], title="TG from gut", xlabel="time (min)", ylabel="TG (mmol/l)")
#p_TG_liv = plot(time, outputs.liver_TG[:], title="TG from liver (VLDL)", xlabel="time (min)", ylabel="TG (mmol/l)")
p_TG_pl = plot(time, outputs.plasma_TG[:], title="Plasma TG", xlabel="time (min)", ylabel="TG (mmol/l)")
p_NEFA_pl = plot(time, outputs.plasma_NEFA[:], title="Plasma NEFA", xlabel="time (min)", ylabel="NEFA (mmol/l)")

plot(p_G_pl, p_I_pl, p_TG_pl, p_NEFA_pl, 
        layout=4, titlefontsize=10, titlefontfamily="Helvetica Bold", fontfamily="Helvetica", 
        titlelocation=:left, size=(400,400), legend=false, linewidth=2, grid=:y,
        left_margin=2mm, bottom_margin=2mm, guidefontfamily="Helvetica Bold", guidefontsize=8, lc=RGB(20/255,55/255,1))

savefig("Figures/Simulation.svg")

## Performing Parameter Estimation

In [ ]:
glc, ins, trg, nfa, bwg, time = SampleData()
individual = 1

m, constants, subject = ModelFromData(glc[individual,:], ins[individual,:], trg[individual,:], nfa[individual,:], bwg[individual])
loss, simulationfunc = LossFunction(m, glc[individual,:], ins[individual,:], trg[individual,:], nfa[individual,:], Int.(time), constants, subject)
p_init = [1.35e-2, 3.80e-3, 5.82e-1, 0.00045, 0.0713, 208.88, 0.0163, 0.0119]
p_lb = [0.005, 0, 0, 0, 0, 60., 0.005, 0]
p_ub = [0.05, 1., 10., 1., 1., 720., 0.1, 1.]
println("Initial loss for individual $(individual): $(loss(p_init))")

In [9]:

res, obj = FitModelSampledLHC(loss, 100, 5, p_lb, p_ub)

In [ ]:
obj

In [ ]:
outputs = simulationfunc(res[argmin(obj),:][1])

glucose_data_plot = scatter(time, glc[individual,:], title="Glucose", labels="Data", ylabel = "[Glucose] (mM)")
plot!(glucose_data_plot, outputs, idxs=2, label="Model")
insulin_data_plot = scatter(time, ins[individual,:], title="Insulin", ylabel = "[Insulin] uIU/ml", labels="")
plot!(insulin_data_plot, outputs, idxs=4, label="")
TG_data_plot = scatter(time, trg[individual,:], title="Triglyceride", ylabel = "[TG] mM", labels="")
plot!(TG_data_plot, outputs, idxs=13, label="")
NEFA_data_plot = scatter(time, nfa[individual,:], title="NEFA", ylabel = "[NEFA] mM", labels="")
plot!(NEFA_data_plot, outputs, idxs=9, label="")


plot(glucose_data_plot, insulin_data_plot, TG_data_plot, NEFA_data_plot, layout=4, titlefontfamily="Helvetica Bold", titlefontsize=11, xlabel="Time (min)", labelfontsize=8, size=(700,450), left_margin=5mm, bottom_margin=5mm)

